In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from xgboost import XGBClassifier
from sklearn import metrics
import seaborn as sns
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from source.read_open_dataset import DatasetOpenDataset, openDatasetParticipants
from source.read_HAR_dataset import DatasetHARUML20
from source.read_pamap2 import DatasetPAMAP2
from source.tserie import TSerie
from source.models.manifold.umap_fl import UMAP_FL
from sklearn import svm

FEATURE_SIZE = 16
UMAP_NEIGHTBORS = 15
N_EPOCHS = 2000
CLASS_LABELS = ['Sedentary', 'Light']

In [12]:
dataset = DatasetPAMAP2(mode='leave-one-subject')

signals=['handAcc16_1', 'handAcc16_2', 'handAcc16_3']
dataset.loadData()

dataset.filterSignals(signals)


In [13]:
info = dataset.participants_info()

In [35]:
# print(info.head())
heights_train = np.array([
    # [info.iloc[idx - 1].Height, info.iloc[idx - 1].Weight, info.iloc[idx - 1].Age, info.iloc[idx - 1]['Subject ID']]
    # [info.iloc[idx - 1].Height, info.iloc[idx - 1].Weight, info.iloc[idx - 1].Age]
    # [info.iloc[idx - 1].Height, info.iloc[idx - 1].Weight]
    [info.iloc[idx - 1].Height]
    for idx in dataset.I_train
])
heights_test = np.array([
    # [info.iloc[idx - 1].Height, info.iloc[idx - 1].Weight, info.iloc[idx - 1].Age, info.iloc[idx - 1]['Subject ID']]
    # [info.iloc[idx - 1].Height, info.iloc[idx - 1].Weight, info.iloc[idx - 1].Age]
    # [info.iloc[idx - 1].Height, info.iloc[idx - 1].Weight]
    [info.iloc[idx - 1].Height]
    for idx in dataset.I_test
])

# weigh_train = np.array([info.iloc[idx - 1].Height for idx in dataset.I_train])
# heights_test = np.array([info.iloc[idx - 1].Height for idx in dataset.I_test])

In [36]:
ts_train = TSerie(dataset.X_train, dataset.Int_train)
ts_test = TSerie(dataset.X_test, dataset.Int_test)


# ts_train.center()
# ts_test.center()

ts_train.folding_features_v2()
ts_test.folding_features_v2()


model = UMAP_FL(n_components=32,n_neighbors=15, n_epochs=2000)
train_feat = model.fit_transform(ts_train.features, y=ts_train.y)
test_feat = model.transform(ts_test.features)

In [22]:
train_feat.shape
# heights_train.shape

(8312, 35)

In [37]:
# train_feat = np.concatenate([train_feat, np.expand_dims(heights_train, axis=1)], axis = 1)
# test_feat = np.concatenate([test_feat, np.expand_dims(heights_test, axis=1)], axis = 1)
train_feat = np.concatenate([train_feat, heights_train], axis = 1)
test_feat = np.concatenate([test_feat, heights_test], axis = 1)

In [38]:
dataset.I_train

array([2, 2, 2, ..., 8, 8, 8], dtype=int32)

In [39]:
# clf = XGBClassifier(tree_method='gpu_hist', predictor='gpu_predictor')
clf = svm.SVC()
train_y = ts_train.y
test_y = ts_test.y
clf.fit(train_feat, train_y)
train_pr = clf.predict(train_feat)
test_pr = clf.predict(test_feat)
# predicted_classes.append(test_pr)
# real_classes.append(test_y)


# Save metrics
train_f1 = metrics.f1_score(train_y, train_pr, average='weighted')
test_f1 = metrics.f1_score(test_y, test_pr, average='weighted')
train_bacc = metrics.balanced_accuracy_score(train_y, train_pr)
test_bacc = metrics.balanced_accuracy_score(test_y, test_pr)
train_acc = metrics.accuracy_score(train_y, train_pr)
test_acc = metrics.accuracy_score(test_y, test_pr)

print(test_bacc)
print(test_f1)

# print(rain_bacc)
# print(test_f1)

# saver.addLog('train', 'F1-score', train_f1)
# saver.addLog('test', 'F1-score', test_f1)
# saver.addLog('train', 'Balanced Accuracy', train_bacc)
# saver.addLog('test', 'Balanced Accuracy', test_bacc)
# saver.addLog('train', 'Accuracy', train_acc)
# saver.addLog('test', 'Accuracy', test_acc)

0.43772681192621266
0.4936094028209468


In [17]:
X = np.concatenate([train_feat, test_feat], axis = 0)
I = np.concatenate([dataset.I_train, dataset.I_test], axis = 0)
y = np.concatenate([dataset.y_train, dataset.y_test], axis = 0)
Int = np.concatenate([dataset.Int_train, dataset.Int_test], axis = 0)

In [11]:
# dataset = {
#     'X': X,
#     'I': I,
#     'y': y,
#     'Int': Int,
# }

# np.save('pamap2_dataset.npy', dataset)